In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from lightning_utils import *
from utils import *
from models import (TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, 
                    ComplexTorchMLP, ComplexSymPyModule, complex_mse)
from models import real2cplx, ComplexAutoEncoder, complex_ae_loss
from preprocess import *

# Model selection
# from sparsereg.model import STRidge # Need Sklearn 0.22.0 to run this.
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

from madgrad import MADGRAD

from tqdm import trange

You can use npar for np.array


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Adding noise
noise_intensity = 0.01/np.sqrt(2)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
del Exact
if noise_intensity > 0.0:
    Exact_u = perturb(Exact_u, intensity=noise_intensity, noise_type="normal")
    Exact_v = perturb(Exact_v, intensity=noise_intensity, noise_type="normal")
    print("Perturbed Exact_u and Exact_v with intensity =", float(noise_intensity))

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u.T)
v_star = to_column_vector(Exact_v.T)

N = 500
idx = np.random.choice(X_star.shape[0], N, replace=False)
# idx = np.arange(N) # Just to have an easy dataset for experimenting

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)

# No need to use utils.group_diff to speed up the computation
feature_names = ['hf', '|hf|', 'h_xx']

### Loading data code here ###
print("Loading pre-calculated data for reproducibility")
X_train = to_tensor(np.load("./tmp_files/X_train_500+500samples.npy"), True)[:N, :]
u_train, v_train = dimension_slicing(to_tensor(np.load("./tmp_files/uv_train_500samples.npy"), False))
u_train = u_train[:N, :]
v_train = v_train[:N, :]
h_train = torch.complex(u_train, v_train)
### ----- ###

You're running on cpu
Perturbed Exact_u and Exact_v with intensity = 0.0070710678118654745
Loading pre-calculated data for reproducibility


In [3]:
cn1 = 0.002494+1.002397*1j
cn2 = 0.003655+0.500415*1j
cns = [cn1, cn2]

In [4]:
# Type the equation got from the symbolic regression step
# No need to save the eq save a pickle file before
program1 = "X0*X1"
pde_expr1, variables1,  = build_exp(program1); print(pde_expr1, variables1)

program2 = "X2"
pde_expr2, variables2,  = build_exp(program2); print(pde_expr2, variables2)

mod = ComplexSymPyModule(expressions=[pde_expr1, pde_expr2], complex_coeffs=cns); mod.train()

X0*X1 {X1, X0}
X2 {X2}


ComplexSymPyModule(
  (sympymodule): SymPyModule(expressions=(X0*X1, X2))
)

In [5]:
class ComplexPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None):
        super(ComplexPINN, self).__init__()
        self.model = model
        self.callable_loss_fn = loss_fn
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        if self.scale and (self.lb is None or self.ub is None): 
            print("Please provide thw lower and upper bounds of your PDE.")
            print("Otherwise, there will be error(s)")
        self.diff_flag = diff_flag(self.index2features)
        
    def forward(self, x, t):
        H = torch.cat([x, t], dim=1)
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, x, t, y_input, update_network_params=True, update_pde_params=True):
        total_loss = []
        grads_dict, u_t = self.grads_dict(x, t)
        # MSE Loss
        if update_network_params:
            mse_loss = complex_mse(grads_dict['X'+self.feature2index['hf']], y_input)
            total_loss.append(mse_loss)
        # PDE Loss
        if update_pde_params:
            l_eq = complex_mse(self.callable_loss_fn(grads_dict), u_t)
            total_loss.append(l_eq)
            
        return total_loss
    
    def grads_dict(self, x, t):
        uf = self.forward(x, t)
        u_t = complex_diff(uf, t)
        
        ### PDE Loss calculation ###
        # Without calling grad
        derivatives = {}
        for t in self.diff_flag[0]:
            if t=='hf': 
                derivatives['X'+self.feature2index[t]] = cplx2tensor(uf)
                derivatives['X1'] = (uf.real**2+uf.imag**2)+0.0j
            elif t=='x': derivatives['X'+self.feature2index[t]] = x
        # With calling grad
        for t in self.diff_flag[1]:
            out = uf
            for c in t:
                if c=='x': out = complex_diff(out, x)
                elif c=='t': out = complex_diff(out, t)
            derivatives['X'+self.feature2index['h_'+t[::-1]]] = out
        
        return derivatives, u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    # Must ensure that the implementation of neural_net_scale is consistent
    # and hopefully correct
    # also, you might not need this function in some datasets
    def neural_net_scale(self, inp): 
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [6]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )
complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


In [7]:
# Pretrained model
semisup_model_state_dict = cpu_load("./saved_path_inverse_nls/NLS_complex_model_500labeledsamples_jointtrainwith500unlabeledsamples.pth")
parameters = OrderedDict()

# Filter only the parts that I care about renaming (to be similar to what defined in TorchMLP).
inner_part = "network.model."
for p in semisup_model_state_dict:
    if inner_part in p:
        parameters[p.replace(inner_part, "")] = semisup_model_state_dict[p]
complex_model.load_state_dict(parameters)

pinn = ComplexPINN(model=complex_model, loss_fn=mod, index2features=feature_names, scale=False, lb=lb, ub=ub)

In [8]:
ct_X_train = real2cplx(X_train)
ae = ComplexAutoEncoder()
opt = torch.optim.Adam(ae.parameters(), lr=1e-3)
for i in trange(10000):
    opt.zero_grad()
    ae.compute_loss(ct_X_train).backward(retain_graph=True)
    opt.step()
ae.eval()
recon_X = ae(ct_X_train); print(complex_mse(recon_X, ct_X_train).item())

100%|██████████| 10000/10000 [00:42<00:00, 233.35it/s]

9.653432243794668e-06


In [10]:
def closure():
    global X_train, h_train, ct_X_train
    if torch.is_grad_enabled(): optimizer2.zero_grad(set_to_none=True)
    denoised_X = ae(ct_X_train)
    losses = pinn.loss(denoised_X.real[:, 0:1], denoised_X.real[:, 1:2], h_train, update_network_params=True, update_pde_params=True)
    l = sum(losses)+0.1*complex_ae_loss(denoised_X, ct_X_train, include_l1=ae.l1_strength)
    if l.requires_grad: l.backward(retain_graph=True)
    return l

def mtl_closure():
    global X_train, h_train, ct_X_train
    denoised_X = ae(ct_X_train)
    losses = pinn.loss(denoised_X.real[:, 0:1], denoised_X.real[:, 1:2], h_train, update_network_params=True, update_pde_params=True)
    losses.append(0.1*complex_ae_loss(denoised_X, ct_X_train, include_l1=ae.l1_strength))
    n_obj = len(losses)
    updated_grads = []
    
    for i in range(n_obj):
        optimizer1.zero_grad()
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in list(ae.parameters()) + list(pinn.parameters()):
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(list(ae.parameters()) + list(pinn.parameters())): 
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    return sum(losses)

In [11]:
epochs1, epochs2 = 200, 50
# TODO: Save best state dict and training for more epochs.
optimizer1 = MADGRAD([{'params':ae.parameters()}, {'params':pinn.parameters()}], lr=1e-7)
optimizer1.param_groups[0]['lr'] = 1e-6
optimizer1.param_groups[1]['lr'] = 1e-7
ae.train(); pinn.train(); best_train_loss = 1e6

print('1st Phase optimization using Adam with PCGrad gradient modification')
for i in range(epochs1):
    optimizer1.step(mtl_closure)
    if (i % 10) == 0 or i == epochs1-1:
        l = mtl_closure()
        print("Epoch {}: ".format(i), l.item())

1st Phase optimization using Adam with PCGrad gradient modification
Epoch 0:  0.011880531907081604
Epoch 10:  0.017022453248500824
Epoch 20:  0.03890106454491615
Epoch 30:  0.008753363974392414
Epoch 40:  0.013701079413294792
Epoch 50:  0.009707205928862095
Epoch 60:  0.02364489994943142
Epoch 70:  0.09004607051610947
Epoch 80:  0.007876990363001823
Epoch 90:  0.07744119316339493
Epoch 100:  0.00961778312921524
Epoch 110:  0.006582675967365503
Epoch 120:  0.005786324851214886
Epoch 130:  0.005736968480050564
Epoch 140:  0.005169949494302273
Epoch 150:  0.005355095956474543
Epoch 160:  0.004800757393240929
Epoch 170:  0.004975044168531895
Epoch 180:  0.005715616047382355
Epoch 190:  0.026916684582829475
Epoch 199:  0.0108265969902277


In [12]:
optimizer2 = torch.optim.LBFGS(list(ae.parameters()) + list(pinn.parameters()), lr=1e-1, max_iter=500, max_eval=int(500*1.25), history_size=300, line_search_fn='strong_wolfe')
ae.train(); pinn.train()

print('2nd Phase optimization using LBFGS')
for i in range(epochs2):
    optimizer2.step(closure)
    if (i % 5) == 0 or i == epochs2-1:
        l = closure()
        print("Epoch {}: ".format(i), l.item())

2nd Phase optimization using LBFGS
Epoch 0:  0.0002032721386058256
Epoch 5:  1.6414111087215133e-05
Epoch 10:  1.3251166819827631e-05
Epoch 15:  1.3251166819827631e-05
Epoch 20:  1.3251166819827631e-05
Epoch 25:  1.3251166819827631e-05
Epoch 30:  1.3251166819827631e-05
Epoch 35:  1.3251166819827631e-05
Epoch 40:  1.3251166819827631e-05
Epoch 45:  1.3251166819827631e-05
Epoch 49:  1.3251166819827631e-05


In [15]:
# save(pinn, "./saved_path_inverse_nls/noisy_final_finetuned_cpinn.pth")
# save(ae, "./saved_path_inverse_nls/ae.pth")

In [13]:
est_coeffs = pinn.callable_loss_fn.complex_coeffs().detach().numpy().ravel()
grounds = np.array([1j, 0+0.5j])

errs = []
for i in range(len(grounds)):
    err = est_coeffs[i]-grounds[i]
    errs.append(100*abs(err.imag)/abs(grounds[i].imag))
errs = np.array(errs)
errs.mean(), errs.std()

(0.3103017807006836, 0.08345842361450195)

In [16]:
est_coeffs

array([0.00169404+1.0022684j, 0.00364474+0.5019688j], dtype=complex64)

In [ ]:
# Noisy NLS
# errs (0.3103017807006836, 0.08345842361450195)
# est_coeffs array([0.00169404+1.0022684j, 0.00364474+0.5019688j], dtype=complex64)